In [1]:
# Here I am going to play around with Kernels, noise, mean functions and hyperparameters
import numpy as np
import plotly.graph_objects as go
from gpcam import GPOptimizer
def plot(x,y,z,data = None):
    fig = go.Figure()
    fig.add_trace(go.Surface(x = x, y = y, z=z))
    if data is not None:
        fig.add_trace(go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2],
                                   mode='markers',marker=dict(size=12,
                                                              color=data[:,2],         # Set color equal to a variable
                                                              #colorscale='Viridis',   # Choose a colorscale
                                                              opacity=0.8)
                                   )
        )

    fig.update_layout(title='Plot', autosize=True,
                  width=800, height=800,
                  margin=dict(l=65, r=50, b=65, t=90),scene=dict(xaxis_title='Temperature', yaxis_title='Cycle', zaxis_title='Energy'))


    fig.show()

## Booth function
$$ 
f(x_1, x_2) = (x_1 + 2x_2 - 7)^2 + (2x_1 + x_2 -5)^2
$$

In [ ]:
# Here we choose to do take the range of x and y from -10 to 10 because that is the most interesting part of the function

x1 = np.arange(-10, 10, 0.1)
x2 = np.arange(-10, 10, 0.1)

X1, X2 = np.meshgrid(x1, x2)

y = (X1 + 2*X2 - 7)**2 + (2*X1 + X2 - 5)**2

#Plot the results
plot(X1, X2, y)

In [116]:
# Here we chose the x 1 values to be -5, 0 and 5 because we deem it the most descriptive choise of 3 values for the range (-10,10), same for (-10,10)

array1 = np.column_stack((np.full((100,), -5), np.arange(-10, 10, 0.2)))
array2 = np.column_stack((np.full((100,), 0), np.arange(-10, 10, 0.2)))
array3 = np.column_stack((np.full((100,), 5), np.arange(-10, 10, 0.2)))

# here, x_data is actually x and y, and the y_data a few lines below is the z data
x_data = np.vstack((array1, array2, array3))

# x1 is x data and x2 is y data, y_data is z data
x1 = x_data[:,0]
x2 = x_data[:,1]

y_data = (x1 + 2*x2 - 7)**2 + (2*x1 + x2 - 5)**2


error = np.random.normal(loc=200, scale=50, size=y_data.shape)
y_data = y_data + error

In [117]:
x_data.shape

(300, 2)

In [60]:
# Define the grid size
n = 100

# Design Space Limits, here we chose (-10,10) for both like we did previously
c_low = -10
c_high = 10
temp_low = -10
temp_high = 10

# Create a design space
y_space = np.linspace(c_low, c_high, n)
x_space = np.linspace(temp_low, temp_high, n)
x_space, y_space = np.meshgrid(x_space, y_space)

# Reshape the arrays into a 2-column array with 10000 rows
my_space = np.vstack((x_space.reshape(-1), y_space.reshape(-1))).T

In [61]:
def get_distance_matrix(x1,x2):
    d = np.zeros((len(x1),len(x2)))
    for i in range(x1.shape[1]):
        d += (x1[:,i].reshape(-1, 1) - x2[:,i])**2
    return np.sqrt(d + 1e-16)

def mean(x, hps):
    y = -(x[:,1]/hps[5])**2 - hps[6]*x[:,1]*(hps[7] - x[:,0]) 
    return y

In [100]:
#####################
## Kernel function ##
#####################

def rational_quadratic_kernel(x1, x2, hps):
    length_scales = hps[2]
    alpha = hps[1]
    signal_variance = hps[0]
    
    # Compute squared distance
    diff = (x1[:, None, :] - x2[None, :, :]) / length_scales
    dist_sq = np.sum(diff ** 2, axis=-1)
    
    # Compute covariance
    cov_matrix = signal_variance ** 2 * (1 + 0.5 * dist_sq / alpha) ** (-alpha)
    
    return cov_matrix

def linear_kernel(x1, x2, hps):
    weights = hps[0]
    bias = hps[1]
    
    # Compute the dot product
    cov_matrix = (x1 @ (x2.T * weights)).sum(axis=-1) + bias
    
    return cov_matrix

def periodic_kernel(x1, x2, hps):

    # length_scales = hps[2]
    # periodicity = hps[1]
    # signal_variance = hps[0]
    # diff = np.pi * (x1[:, None, :] - x2[None, :, :]) / periodicity
    
    # cov_matrix = signal_variance ** 2 * np.exp(-2 * np.sum(np.sin(diff) ** 2 / length_scales ** 2, axis=-1))
    # return cov_matrix

    length_scales = hps[1]
    signal_variance = hps[0]
    
    diff = (x1[:, None, :] - x2[None, :, :]) / length_scales
    dist_sq = np.sum(diff ** 2, axis=-1)
    
    return signal_variance ** 2 * np.exp(-0.5 * dist_sq)

Rational Quadratic Kernel: $k(\mathbf{x}_1, \mathbf{x}_2) = \sigma^2 \left(1 + \frac{\|\mathbf{x}_1 - \mathbf{x}_2\|^2}{2\alpha l^2}\right)^{-\alpha}$

Periodic Kernel: $ k(\mathbf{x}_1, \mathbf{x}_2) = \sigma^2 \exp\left(-\frac{2}{l^2} \sum_{d=1}^{D} \sin^2\left(\frac{\pi (\mathbf{x}_{1d} - \mathbf{x}_{2d})}{p}\right)\right) $


In [83]:
#####################
## Noise functions ##
#####################

def s(x, hps):
    slope = hps[3]
    offset = hps[4]
    o = slope * x[:, 0] + offset
    
    # Ensure positive diagonal values
    o[o <= 0] = np.abs(o[o <= 0]) + 1e-6
    
    return np.diag(o)


In [85]:
####################
## Mean functions ##
####################

# If None is provided, fvgp.GP._default_mean_function is used, which is the average of the y_data.

def mean(x, hps):
    y = -(x[:,1]/hps[5])**2 - hps[6]*x[:,1]*(hps[7] - x[:,0]) 
    return y

def zero_mean_function(x, hps):

    array_length = x.shape[0]
    zero_array = np.zeros(array_length)
    return zero_array

def sinusoidal_mean_function(x, hps):
    
    omega = hps[5]    # This is the freqency omega
    alpha = hps[6]    # this is the amplitude kind of like in our sine wave
    
    # Calculate the mean as a sum of sinusoidal functions for each dimension
    mean_values = alpha * np.sum(np.sin(omega * x[:, :2]), axis=1)
    
    return mean_values


In [94]:
#############################
## Setting Hyperparameters ##
#############################

# hps[0] 1, 2 for kernel. 3,4 for noise, 5,6,7 for meanf
bounds = np.array([[0.001,1000000]    # in kernel, signal variance
                   ,[0.001,100]   # in kernel, length scale
                   ,[0.001,100]   # in kernel, length scale 2
                   ,[0.1,10]      # s, slope
                   ,[0.1,10]      # s, offset
                   ,[0, 2500]     # mean, 
                   ,[0, 2500]     # mean
                   ])

my_trained_hps = np.array([0.01      # in kernel
                           , 0.01    # in kernel
                           , 0.01    # in kernel
                           , 1       # s
                           , 1       # s
                           , 800      # mean
                           , 40    # mean
                           ])

noise_hps = np.array([0.01      # in kernel
                           , 0.01    # in kernel
                           , 0.01    # in kernel
                           , 1       # s
                           , 1       # s
                           , 50 # extra noise hps
                           , 800      # mean
                           , 40    # mean
                           ])

In [104]:
def create_and_train(x_data, y_data, my_trained_hps, hps_bounds, kernel_function=None, noise_function=None, mean_function=None):

    my_gpo = GPOptimizer(x_data, y_data,
                      init_hyperparameters=my_trained_hps,
                      # hyperparameter_bounds=hps_bounds,
                      # gp_kernel_function= kernel_function,
                      gp_mean_function= mean_function,
                      # gp_noise_function= noise_function
                      )
    my_gpo.train(hyperparameter_bounds=bounds)

    f = my_gpo.posterior_mean(my_space)["f(x)"]
    v = my_gpo.posterior_covariance(my_space, add_noise=True)["v(x)"]

    f_re = f.reshape(100,100)

    plot(x_space, y_space, f_re, data = np.column_stack([my_gpo.x_data, my_gpo.y_data]))

    return my_gpo

In [ ]:
# s noise function
my_gpo = create_and_train(x_data, y_data, 
                 my_trained_hps = my_trained_hps, 
                 hps_bounds = my_trained_hps, 
                 # kernel_function = rational_quadratic_kernel,
                 # mean_function =,
                 noise_function = s
                 )

In [ ]:
# Rational Quadratic Kernel
my_gpo = create_and_train(x_data, y_data, 
                 my_trained_hps = my_trained_hps, 
                 hps_bounds = bounds, 
                 kernel_function = rational_quadratic_kernel,
                 # mean_function =,
                 # noise_function =,
                 )

In [ ]:
# Periodic Kernel
my_gpo = create_and_train(x_data, y_data, 
                 my_trained_hps = my_trained_hps, 
                 hps_bounds = bounds, 
                 kernel_function = periodic_kernel,
                 # mean_function =,
                 # noise_function =,
                 )

In [ ]:
# Linear kernel
my_gpo = create_and_train(x_data, y_data, 
                 my_trained_hps = my_trained_hps, 
                 hps_bounds = bounds, 
                 kernel_function = linear_kernel,
                 # mean_function =,
                 # noise_function =,
                 )

In [ ]:
# Default kernel
my_gpo = my_gpo = create_and_train(x_data, y_data, 
                 my_trained_hps = my_trained_hps, 
                 hps_bounds = bounds, 
                 # kernel_function = ,
                 # mean_function =,
                 # noise_function =,
                 )

In [ ]:
# Zero mean function and default kernel

# (make sure to change the create_and_train function)

my_gpo = create_and_train(x_data, y_data,
                 my_trained_hps = my_trained_hps, 
                 hps_bounds = bounds, 
                 # kernel_function = ,
                 mean_function = zero_mean_function,
                 # noise_function =,
                 ) 

In [105]:
# Sinusoidal mean
my_gpo = create_and_train(x_data, y_data,
                 my_trained_hps = my_trained_hps, 
                 hps_bounds = bounds, 
                 # kernel_function = ,
                 mean_function = sinusoidal_mean_function,
                 # noise_function =,
                 ) 